### OBJECTIVE
#### The purpose of this notebook is to calculate re-order point for an online retail store across all sku's. The re-order point is calculated after ABC analysis and considering the lead time variability.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install inventorize3

In [ ]:
import inventorize3 as inv
import datetime

In [ ]:
retail=pd.read_csv('../input/online-retail-ii-uci/online_retail_II.csv')

In [ ]:
## dropping duplicates

retail= retail.drop_duplicates()

In [ ]:
retail.isna().sum()

In [ ]:
## dropping null rows

retail=retail.dropna(axis=0)

In [ ]:
retail.info()

In [ ]:
## checking date column of any unwanted component i.e hh:mm:ss
retail.InvoiceDate.head()

In [ ]:
## converting to datetime format

retail['InvoiceDate']= pd.to_datetime(retail['InvoiceDate'])

In [ ]:
## extracting date

retail['date']= retail.InvoiceDate.dt.strftime("%Y-%m-%d")

In [ ]:
retail['date'].head()

In [ ]:
## convert to datetime

retail['date']= pd.to_datetime(retail['date'])

### Extracting last 4 month data

In [ ]:
max(retail.date)

In [ ]:
last_four = retail[retail.date > "2011-08-01"]

In [ ]:
last_four.head()

In [ ]:
## adding the revenue column

last_four['revenue']=last_four['Quantity']* last_four['Price']

In [ ]:
last_four.head()

### Calculating avg. demand and standard deviation per sku on each day

In [ ]:
a = last_four.groupby(['date','Description']).agg(total_daily = ('Quantity',np.sum),
                                                   total_revenue = ('revenue',np.sum)).reset_index()

In [ ]:
a.head()

In [ ]:
## calc. mean and sd of total_daily

grouped = a.groupby("Description").agg(average= ('total_daily',np.mean),
                                       sd = ('total_daily','std'),
                                       total_sales=('total_daily',np.sum),
                                       total_revenue=('total_revenue',np.sum)).reset_index()

In [ ]:
grouped.head()

### ABC Analysis

In [ ]:
for_abc = inv.productmix(grouped['Description'],grouped['total_sales'],grouped['total_revenue'])

In [ ]:
for_abc

### Result validation

In [ ]:
for_abc.product_mix.value_counts()

#### Majority of products are C_C i.e slow moving, low value items and very few C_A i.e slow moving, high margin items.

In [ ]:
## defining lead time and error margin on lead time i.e SD

lead_time =12
sd_leadtime =2

#### Defining the service level for ABC sku's, service level range from 95% for high value to 70% for low value.

In [ ]:
## making a dictionary list for service level

mapping ={"A_A":0.95, "A_B":0.95, "A_C":0.95,
          "C_A":0.8, "C_B":0.8, "C_C":0.7,
           "B_A":0.7, "B_B":0.7, "B_C":0.75}

In [ ]:
for_abc['service_level']= for_abc.product_mix.map(mapping)

In [ ]:
for_abc.head()

### Re-Order Point

In [ ]:
## extracting sku and service level column

abcd = for_abc[['skus','service_level']]

In [ ]:
## merging abcd and grouped df

for_reorder=pd.merge(grouped, abcd, how='left', left_on='Description', right_on='skus')

In [ ]:
for_reorder.head()

### Calculating re-order point with lead-time variablity of 2

In [ ]:
## create empty df

df_ltv = pd.DataFrame()

In [ ]:
for_reorder.columns

In [ ]:
for i in range(for_reorder.shape[0]):
    ordering_point = inv.reorderpoint_leadtime_variability(int(for_reorder.loc[i,'average']),
                                      for_reorder.loc[i,'sd'], 12,2,  # lead_time is 12 & variablity is 2
                                      for_reorder.loc[i,'service_level'])
    as_data = pd.DataFrame(ordering_point,index=[0])
    as_data['Descrition']= for_reorder.loc[i,'Description']
    df_ltv = pd.concat([df_ltv, as_data], axis=0)

In [ ]:
df_ltv

#### We now have re-order point, sigma dl, demand lead time and safety factor for all skus. Now we join this data with for_reorder df.

In [ ]:
all_data= pd.merge(for_reorder, df_ltv, how='left', left_on='Description', right_on='Descrition')

In [ ]:
all_data.head()

### Calculating Safety Stock

In [ ]:
all_data['safety_stock']= all_data['reorder_point'] - all_data['demandleadtime']

In [ ]:
all_data.head()

### Visulalizing the safety stock on each service level with respect to SD

In [ ]:
## removing the outlier i.e fast moving product with high average as it is going to impact the visulaization

all_data[all_data.safety_stock == max(all_data.safety_stock)]

In [ ]:
## the above sku has a very high avg and sd therefore we'll remove it from visualization

all_data=all_data[all_data.safety_stock != max(all_data.safety_stock)]

In [ ]:
import seaborn as sns

sns.scatterplot(x='sd', y='safety_stock', hue='service_level', data= all_data)

#### As we can see from the above plot, the higher the service level the more safety stock is required.

In [ ]:
sns.scatterplot(x='demandleadtime', y='safety_stock', hue='service_level', data= all_data)

#### As we can see that most of the demand during lead time is within 2000 saftey_stock except for few high service level items.